# Analyse sequentielle sur des données de métro

Objectif: prédire la ville à partir des observations

Le jeu de données du métro de Hangzhou décrit le flux entrant et sortant de  80 stations de la ville agrégée par quart d'heure entre $5h30$ et $23h30$ chaque jour. Deux tenseurs sont dans l'archive, un d'apprentissage et l'autre de test. Ils sont  de taille $D\times T \times S \times 2$ avec $D$ le nombre de jour, $T=73$ les tranches successives de quart d'heure entre $5h30$ et $23h30$, $S=80$ le nombre  de stations et les flux entrant et sortant pour la dernière dimension.

On va travailler sur un **sous-échantillon pour simplifier les choses**: ce sous-échantillon est paramétré dans les boites ci-dessous.

In [1]:
from utils import RNN, device,SampleMetroDataset
import torch
from torch.utils.data import DataLoader

In [2]:
import numpy as np
import torch.nn as nn
import torch.optim
from torch.utils.data import DataLoader
import logging
import tensorboard
from torch.utils.tensorboard import SummaryWriter
import time
from itertools import chain
logging.basicConfig(level=logging.INFO)


In [3]:
from pathlib import Path
from IPython.display import display, HTML
from torch.utils.tensorboard import SummaryWriter

# Chemin vers TensorBoard
TB_PATH = "/tmp/logs"

# TENSORBOARD V2 (outside notebook => Navigateur)
# usage externe de tensorboard: (1) lancer la commande dans une console; (2) copier-coller l'URL dans un navigateur
display(HTML("<h2>Informations</h2><div>Pour visualiser les logs, tapez la commande : </div>"))
print(f"tensorboard --logdir {Path(TB_PATH).absolute()}")
print("Une fois la commande lancer dans la console, copier-coller l'URL dans votre navigateur")

tensorboard --logdir /tmp/logs
Une fois la commande lancer dans la console, copier-coller l'URL dans votre navigateur


## A. Chargement d'un sous-échantillon des données

In [4]:
# Nombre de stations utilisé 
CLASSES = 10
#Longueur des séquences 
LENGTH = 20
# Dimension de l'entrée (1 (in) ou 2 (in/out))
DIM_INPUT = 2
#Taille du batch
BATCH_SIZE = 32

PATH = "./data/"

matrix_train, matrix_test = torch.load(open(PATH+"hzdataset.pch","rb"))
ds_train = SampleMetroDataset(matrix_train[:, :, :CLASSES, :DIM_INPUT], length=LENGTH)
ds_test = SampleMetroDataset(matrix_test[:, :, :CLASSES, :DIM_INPUT], length = LENGTH, stations_max = ds_train.stations_max)
data_train = DataLoader(ds_train,batch_size=BATCH_SIZE,shuffle=True)
data_test = DataLoader(ds_test, batch_size=BATCH_SIZE,shuffle=False)


In [5]:
# analyse des données
# quelques proposition... Mais à vous de jouer

print(len(ds_train))
print(ds_train[0])
print(ds_train[0][0].size())

cl = [ds_train[i][1] for i in range(len(ds_train))]

print(np.unique(cl))
print(ds_train[1][0].sum(0))

9540
(tensor([[0.0000, 0.0040],
        [0.0021, 0.0066],
        [0.0000, 0.0370],
        [0.0021, 0.0529],
        [0.1874, 0.0820],
        [0.0695, 0.1045],
        [0.0737, 0.1984],
        [0.0821, 0.1918],
        [0.0821, 0.1918],
        [0.0968, 0.2421],
        [0.1053, 0.2500],
        [0.1095, 0.2659],
        [0.1621, 0.2526],
        [0.1937, 0.3135],
        [0.1937, 0.2884],
        [0.2168, 0.3836],
        [0.2674, 0.3082],
        [0.2168, 0.3889],
        [0.2632, 0.3161],
        [0.3663, 0.3796]]), 0)
torch.Size([20, 2])
[0 1 2 3 4 5 6 7 8 9]
tensor([3.7095, 8.4339])


In [6]:

# paramértage par défaut
dim_input=2
epochs=100
batch_size=32
length=20
latent=10
classes=10


In [7]:
rnn = RNN(dim_input,latent) # cf code dans utils.py
decoder = nn.Linear(latent,classes)
loss = torch.nn.CrossEntropyLoss()
optim = torch.optim.Adam(chain(rnn.parameters(),decoder.parameters()),lr=0.0001)
writer = SummaryWriter(TB_PATH+"/predictStations-"+time.asctime())


## B. Apprentissage

L'enjeu est de faire rentrer ce réseau de neurones dans la boucle d'apprentissage classique développée dans les séances précédante.

1. Récupérer une boucle d'apprentissage standard dans un TP précédent
2. Jouer avec le modèle rnn pour comprendre ses entrées et ses sorties
    - vérifier les dimensions de sortie du réseau
    - choisir où brancher le décodeur
3. Itérer jusqu'à réussir à lancer l'apprentissage

Vérifier les performances directement sur tensorboard au bout de l'apprentissage

In [8]:
# boucle standard d'apprentissage
# 


def train(rnn, decoder, epochs):
    rnn = rnn.to(device)
    decoder = decoder.to(device)
## TODO 

In [9]:
train(rnn, decoder, epochs)

INFO:root:Iteration 0
INFO:root:loss train : 2.335583 -- 0.900293
INFO:root:loss test : 2.312457 --0.900054
INFO:root:Iteration 1
INFO:root:loss train : 2.299029 -- 0.898620
INFO:root:loss test : 2.280795 --0.887931
INFO:root:Iteration 2
INFO:root:loss train : 2.271045 -- 0.884197
INFO:root:loss test : 2.251405 --0.847791
INFO:root:Iteration 3
INFO:root:loss train : 2.236055 -- 0.843645
INFO:root:loss test : 2.199703 --0.823815
INFO:root:Iteration 4
INFO:root:loss train : 2.159901 -- 0.826505
INFO:root:loss test : 2.111423 --0.830550
INFO:root:Iteration 5
INFO:root:loss train : 2.098709 -- 0.839674
INFO:root:loss test : 2.078041 --0.839709
INFO:root:Iteration 6
INFO:root:loss train : 2.072749 -- 0.839360
INFO:root:loss test : 2.056861 --0.838901
INFO:root:Iteration 7
INFO:root:loss train : 2.054338 -- 0.836957
INFO:root:loss test : 2.040826 --0.828664
INFO:root:Iteration 8
INFO:root:loss train : 2.038416 -- 0.823892
INFO:root:loss test : 2.024000 --0.812231
INFO:root:Iteration 9
INFO:r

RNN(
  (encoder): Linear(in_features=2, out_features=10, bias=True)
  (latent): Linear(in_features=10, out_features=10, bias=True)
)

## C. Prediction d'affluence

L'objectif de cette partie est de faire de la prédiction de séries temporelles : à partir d'une séquence de flux de longueur $t$ pour l'ensemble des stations du jeu de données, on veut prédire le flux à $t+1$, $t+2$, $\ldots$. Vous entraînerez un RNN commun à toutes les stations qui prend une série dans $\mathbb{R}^{n\times 2}$ et  prédit une série dans $\mathbb{R}^{n\times 2}$.

Que doit-on changer au modèle précédent ? Quel coût est dans ce cas plus adapté que la cross-entropie ? 

 Faire les expériences en faisant varier l'horizon de prédiction (à $t+2$, etc.) et la longueur des séquences en entrée. Vous pouvez comme précédemment  considérer d'abord que le flux entrant, puis le flux entrant et sortant.


Dans ce contexte de réseau \textit{many-to-many}, la supervision peut se faire à chaque étape de la séquence sans attendre la fin de la séquence. La rétro-propagation n'est faîte qu'une fois que toute la séquence a été vue, mais à un instant $t$, le gradient prend en compte l'erreur à ce moment (en fonction de la supervision du décodage) mais également l'erreur des pas de temps d'après qui est cumulée. 

In [11]:
# paramértage par défaut
dim_input=2
epochs=100
batch_size=32
length=20
latent=10
classes=10

stations = 20
length=20
length_fc=10

In [12]:
from utils import RNN, device,  ForecastMetroDataset

matrix_train, matrix_test = torch.load(open(PATH+"hzdataset.pch","rb"))
data_train = DataLoader(ForecastMetroDataset(matrix_train[:,:,:stations,:dim_input],length=length),batch_size=batch_size,shuffle=True)
data_test = DataLoader(ForecastMetroDataset(matrix_test[:,:,:stations,:dim_input],length=length,stations_max=stations),batch_size=batch_size,shuffle=False)


In [ ]:
# on fournit le code du prédicteur qui permet d'exploiter la structure du réseau de neurones
# et de ré-injecter la sortie prédite pour prédire les valeurs suivantes

# evidemment, le code est dépendant d'un décodeur qu'il faut déinir

def forecast(rnn,decoder,x,h=None,length=10):
    with torch.no_grad():
        if h is None:
            h = rnn.hzero(x.size(1)).to(x.device)
        h = rnn.forward(x,h)[-1]
        x = decoder.forward(h)
        yhat = [x]
        for i in range(length-1):
            x = decoder.forward(rnn.one_step(x,h))
            yhat.append(x)
    return torch.stack(yhat)

In [13]:

# rnn = 
# decoder =
# loss = 

## TODO 

optim = torch.optim.Adam(chain(rnn.parameters(),decoder.parameters()),lr=0.0001)
writer = SummaryWriter(PATH+"/Forecast-"+time.asctime())


In [15]:
def train(rnn, decoder, epochs):
## TODO 
    return rnn

In [16]:
train(rnn, decoder, epochs)

INFO:root:Iteration 0
INFO:root:loss train : 0.148785
INFO:root:loss test : 815.944580
INFO:root:loss test forecast : 759.390625
INFO:root:Iteration 1
INFO:root:loss train : 0.135398
INFO:root:loss test : 815.960754
INFO:root:loss test forecast : 759.082947
INFO:root:Iteration 2
INFO:root:loss train : 0.124324
INFO:root:loss test : 816.228821
INFO:root:loss test forecast : 758.646912
INFO:root:Iteration 3
INFO:root:loss train : 0.115152
INFO:root:loss test : 816.694824
INFO:root:loss test forecast : 758.198303
INFO:root:Iteration 4
INFO:root:loss train : 0.106993
INFO:root:loss test : 817.152039
INFO:root:loss test forecast : 757.779175
INFO:root:Iteration 5
INFO:root:loss train : 0.099838
INFO:root:loss test : 817.403931
INFO:root:loss test forecast : 757.440735
INFO:root:Iteration 6
INFO:root:loss train : 0.093112
INFO:root:loss test : 817.561829
INFO:root:loss test forecast : 757.176392
INFO:root:Iteration 7
INFO:root:loss train : 0.087045
INFO:root:loss test : 817.452515
INFO:root:

RNN(
  (encoder): Linear(in_features=40, out_features=10, bias=True)
  (latent): Linear(in_features=10, out_features=10, bias=True)
)

# Construction du sujet à partir de la correction

In [ ]:
###  TODO )"," TODO ",\
    txt, flags=re.DOTALL))
f2.close()

### </CORRECTION> ###